## Multi-mode data

Finally we repeat the modelling part of this notebook using in this case multiple modes (fundamental, 1st, and 2nd).

In [1]:
# %matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pylops
from PIL import Image

from functools import partial
from scipy.optimize import minimize, Bounds
from disba import PhaseDispersion

from Dispersion.surfacewaves import *
from Dispersion.dispersionspectra import *
from Dispersion.inversion import *

import ccfj
import scipy
from Dispersion.Dispersion.dispersion import get_dispersion


In [2]:
def get_cpr(thick, vs, period):
    
    true_model = np.vstack([thick, vs*4, vs, np.ones_like(vs)]).T

    # Rayleigh-wave fundamental model dispersion curve 
    pd = PhaseDispersion(*true_model.T)
    cpr = [pd(period, mode=imode, wave="rayleigh") for imode in range(3)]

    return cpr

def random_thick_vs(thick, vs, period, fluctuation_percentage=0.1):
    # 生成浮动值
    random_thick = thick * (1 + fluctuation_percentage * (2 * np.random.rand(len(thick)) - 1))
    random_vs = vs * (1 + fluctuation_percentage * (2 * np.random.rand(len(vs)) - 1))

    try:
        cpr = get_cpr(random_thick, random_vs, period)
        # plt.plot(1/cpr[0][0], cpr[0][1], 'k', lw=2, label='Original')
        return cpr
    except Exception as e:
        print(e)

def get_dshift(nt, dt, nx, dx, nfft, cpr):
    t, x = np.arange(nt)*dt, np.arange(nx)*dx

    # Wavelet
    wav = ormsby(t[:nt//2+1], f=[2, 4, 38, 40], taper=np.hanning)[0][:-1]
    wav = np.roll(np.fft.ifftshift(wav), 20) # apply small shift to make it causal

    # Data
    dshifts, fs, vfs = [], [], []
    for imode in range(3):
        dshift_, f_, vf_ = surfacewavedata(nt, dt, nx, dx, nfft, 
                                        np.flipud(1/cpr[imode][0]), np.flipud(cpr[imode][1]), wav)
        dshifts.append(1./(imode+1)**0.8 * dshift_[np.newaxis])
        fs.append(f_)
        vfs.append(vf_)
    dshift = np.concatenate(dshifts).sum(0)
    return dshift

def park(dshift, dx, dt, cmin, cmax, dc, fmin, fmax):
    f1, c1, img, U, t = get_dispersion(dshift.T, dx, dt, 
                                        cmin, cmax, dc, fmin, fmax)

    return f1, c1, img, U, t

def fj(dshift, dx, dt, cmin, cmax):
    nx, nt = dshift.shape
    x = np.arange(nx)*dx
    f = scipy.fftpack.fftfreq(nt,dt)[:nt//2]
    c = np.linspace(cmin, cmax, 1000)

    out = ccfj.fj_earthquake(dshift,x,c,f,fstride=1,itype=0,func=0)
    
    return f, c, out

def show_fj(f, c, out, fmin, fmax, ii, aa):
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    ax.axis("off")
    ax.imshow(out, aspect='auto', cmap='gray',
            extent=(f.min(), f.max(),c.min(), c.max()),origin='lower')

    ax.margins(0)
    ax.set_xlim(fmin, fmax)
    ax.set_ylim(c.min(), c.max())
    fig.savefig(f'/home/lty/MyProjects/Seismology/diffseis/dataset/demultiple/data_train/data/{aa}{ii:03d}.png', 
                dpi=300,bbox_inches='tight', pad_inches=0)
    plt.close()

def show_label(f, c, out, cpr, fmin, fmax, ii, aa):
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    ax.axis("off")
    ax.imshow(np.zeros_like(out), aspect='auto', cmap='gray',
            extent=(f.min(), f.max(),c.min(), c.max()),origin='lower')
    for imode in range(3):
        ax.plot(np.flipud(1/cpr[imode][0]), 1.e3*np.flipud(cpr[imode][1]), 
                    'white', lw=4)

    ax.margins(0)
    ax.set_xlim(fmin, fmax)
    ax.set_ylim(c.min(), c.max())

    fig.savefig(f'/home/lty/MyProjects/Seismology/diffseis/dataset/demultiple/data_train/labels/{aa}{ii:03d}.png', 
                dpi=300,bbox_inches='tight', pad_inches=0)
    plt.close()


def save_image(image_numpy, image_path, aspect_ratio=1.0):
    """Save a numpy image to the disk

    Parameters:
        image_numpy (numpy array) -- input numpy array
        image_path (str)          -- the path of the image
    """

    image_pil = Image.fromarray(image_numpy)
    h, w, _ = image_numpy.shape

    if aspect_ratio is None:
        pass
    elif aspect_ratio > 1.0:
        image_pil = image_pil.resize((h, int(w * aspect_ratio)), Image.BICUBIC)
    elif aspect_ratio < 1.0:
        image_pil = image_pil.resize((int(h / aspect_ratio), w), Image.BICUBIC)
    image_pil.save(image_path)


In [4]:
import numpy as np
import random
import concurrent.futures


# Frequency axis
fdisp = np.linspace(3, 20, 301)
period = np.flipud(1/fdisp)         # Periods (must be sorted starting with low periods)

# Axes
nt = 600 # number of time samples
dt = 0.008 # time sampling in s
nx = 201 # number of spatial samples
dx = 2 # spatial sampling in m
nfft = 2**10

dc = 1.
cmin, cmax = 50., 800.
fmin, fmax = fdisp.min(), fdisp.max()


# 原始数组
# thick = np.array([0.01, 0.02, 0.03, 0.01])
# vs = np.array([0.2, 0.4, 0.6, 0.8])
fluctuation_percentage = 0.2        # 定义浮动范围百分比

def process_combination(h1, h2, h3, h4, v1, v2, v3, v4, nt, dt, nx, dx, nfft, period, fluctuation_percentage, cmin, cmax, fmin, fmax):
    thick = np.array([h1, h2, h3, h4])
    vs = np.array([v1, v2, v3, v4])
    for ii in range(3):
        try:
            cpr = random_thick_vs(thick, vs, period, fluctuation_percentage)
            dshift = get_dshift(nt, dt, nx, dx, nfft, cpr)
            f, c, out = fj(dshift, dx, dt, cmin, cmax)
            aa = f"{h1:.3f}_{h2:.3f}_{h3:.3f}_{h4:.3f}_{v1:.3f}_{v2:.3f}_{v3:.3f}_{v4:.3f}"
            show_fj(f, c, out, fmin, fmax, ii, aa)
            show_label(f, c, out, cpr, fmin, fmax, ii, aa)
        except Exception as e:
            print(e)
            continue

h4 = 1.0
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for h1 in np.linspace(0.005, 0.02, 3):
        for h2 in np.linspace(0.01, 0.04, 3):
            for h3 in np.linspace(0.02, 0.06, 3):
                # for h4 in np.linspace(0.03, 0.07, 3):
                for v1 in np.linspace(0.1, 0.3, 2):
                    for v2 in np.linspace(0.3, 0.6, 3):
                        for v3 in np.linspace(0.3, 0.7, 3):
                            for v4 in np.linspace(0.7, 0.9, 3):
                                futures.append(executor.submit(process_combination, h1, h2, h3, h4, v1, v2, v3, v4, nt, dt, nx, dx, nfft, period, fluctuation_percentage, cmin, cmax, fmin, fmax))
    concurrent.futures.wait(futures)





array of sample points is empty
array of sample points is empty
array of sample points is empty
array of sample points is empty
array of sample points is empty
array of sample points is empty
array of sample points is empty
array of sample points is empty
array of sample points is empty
array of sample points is empty
failed to find root for fundamental mode
'NoneType' object is not subscriptable
array of sample points is empty
failed to find root for fundamental mode
'NoneType' object is not subscriptable
failed to find root for fundamental mode
'NoneType' object is not subscriptable
array of sample points is empty
array of sample points is empty
failed to find root for fundamental mode
'NoneType' object is not subscriptable
array of sample points is empty
failed to find root for fundamental mode
'NoneType' object is not subscriptable
failed to find root for fundamental mode
'NoneType' object is not subscriptable
array of sample points is empty
failed to find root for fundamental mode

In [5]:
3 ** 7 * 2

4374